<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ENEE439d-TEXTML/TextML/blob/master/input.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/ENEE439d-TEXTML/TextML/blob/master/input.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Data input and text removal testing testing

In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras import layers

import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
filepath = '/content/drive/MyDrive/ml-data/data.csv' #'data.csv'
df= pd.read_csv(filepath)
df1=df.drop(['no','paper'],axis=1)
df1.head()

,cited_paper,label,text
0,A00-2024,0,We analyzed a set of articles and identified s...
1,A00-2024,0,Table 3: Example compressions Compression AvgL...
2,A00-2024,0,5.3 Related works and discussion Our two-step ...
3,A00-2024,0,(1999) proposed a summarization system based o...
4,A00-2024,0,We found that the deletion of lead parts did n...


In [6]:
df1['label'].value_counts()

 0    898
 1     80
-1     21
Name: label, dtype: int64

In [12]:
context=df1['text']
# pp.pprint(context[0])
# pp.pprint(context[1])
# pp.pprint(context[2])
# pp.pprint(context[3])
# pp.pprint(context[4])

re1= "\(((([A-Za-z]+ *)+(, \d+))+(; )*)+\)" # matches author and author, year
re_year=",? \(?\d{4}\)?" # match , {4 digits} which may be wrapped in () 
re_et= "(\w+ *)et al\. ?"+re_year # matches author et al. , year
re_2a= "(\w+ *)and ((\w+ *))?"+re_year # matches author and author, year
re_sep="((; )|( and ))*"# match the '; ' gap or ' and ' gap
re_apa ="\(?("+"(\(?"+re_2a+"|"+re_et+"\)?)"+re_sep+")+"
p1= re.compile(re_apa)
a=p1.search(context[0])
print(a)
a=p1.search(context[2])
print(a)
#p1= re.compile(re_2a)
a=p1.search(context[4])
print(a)
print(a.groups())

<re.Match object; span=(437, 485), match='(Jing and McKeown, 1999; Jing and McKeown, 2000)'>
<re.Match object; span=(109, 158), match='(Mani et al. , 1999) and (Jing and McKeown, 2000)>
<re.Match object; span=(101, 124), match='Jing and McKeown (2000)'>
('Jing and McKeown (2000)', 'Jing and McKeown (2000)', 'Jing ', 'McKeown', 'McKeown', None, None, None, None)


In [13]:
def remove_citation(text,regex=re_apa):
  text1=text
  rem_len=0
  pattern= re.compile(regex)
  while True:
    matches=pattern.search(text1)
    # print(matches)
    if matches == None:
      break

    spn=matches.span()
    text1=text1[0:spn[0]]+text1[spn[1]:-1]
    cit_len=spn[1]-spn[0]
    rem_len+=cit_len
  
  percent_removed=rem_len/len(text)
  return text1,percent_removed 

print(context[5])
remove_citation(context[5],regex=re_apa)

Automatic text summarization approaches have offered reasonably well-performing approximations for identifiying important sentences (Lin and Hovy, 2002; Schiffman et al., 2002; Erkan and Radev, 2004; Mihalcea and Tarau, 2004; Daume III and Marcu, 2006) but, not surprisingly, text (re)generation has been a major challange despite some work on sub-sentential modification (Jing and McKeown, 2000; Knight and Marcu, 2000; Barzilay and McKeown, 2005).


('Automatic text summarization approaches have offered reasonably well-performing approximations for identifiying important sentences Daume  but, not surprisingly, text (re)generation has been a major challange despite some work on sub-sentential modification ',
 0.4209354120267261)

In [23]:
output=df1['text'].apply(lambda x: remove_citation(text=x)) #df['col1'] = df.apply(lambda x: complex_function(x['col1']), axis=1)
df_o = pd.DataFrame(list(output), columns =['clean','p_rem'])
#df_o.head()

df1['text_clean']=df_o['clean']
df1['text_clean_len']=df_o['clean'].apply(len)
df1['p_rem']=df_o['p_rem']
df1.head()

,cited_paper,label,text,text_clean,p_rem,text_clean_len
0,A00-2024,0,We analyzed a set of articles and identified s...,We analyzed a set of articles and identified s...,0.098765,437
1,A00-2024,0,Table 3: Example compressions Compression AvgL...,Table 3: Example compressions Compression AvgL...,0.065903,325
2,A00-2024,0,5.3 Related works and discussion Our two-step ...,5.3 Related works and discussion Our two-step ...,0.308176,109
3,A00-2024,0,(1999) proposed a summarization system based o...,(1999) proposed a summarization system based o...,0.062500,344
4,A00-2024,0,We found that the deletion of lead parts did n...,We found that the deletion of lead parts did n...,0.184000,101


# Remove large samples

In [24]:
df1['text_clean_len'].sort_values()

408        0
36         0
144        0
205        0
822        0
       ...  
907     1001
637     1211
908     2118
909     3574
225    30057
Name: text_clean_len, Length: 999, dtype: int64

In [31]:
df3 = df1.loc[df1['p_rem'] > .75]
df3

,cited_paper,label,text,text_clean,p_rem,text_clean_len
36,A92-1018,0,"(Cutting et al. , 1992)).",,0.960000,0
144,C08-1041,0,"(He et al., 2008).",,0.944444,0
205,C08-1144,0,"(Zollmann et al., 2008).",,0.958333,0
307,D07-1001,0,"(Elhadad et al., 2001; Clarke and Lapata, 2007...",,0.985915,0
356,D07-1013,0,"also McDonald and Nivre, 2007).",also,0.806452,5
393,D07-1014,0,"(Smith and Smith, 2007)).",),0.920000,1
398,D07-1014,0,"(McDonald and Satta, 2007; Smith and Smith, 20...",,0.980000,0
408,D07-1020,0,"(Mann and Yarowsky, 2003; Chen and Martin, 200...",,0.986667,0
480,D07-1061,0,This is similartothegraphconstructionmethodofH...,This is Rao et al,0.785714,17
585,D07-1090,0,"(Emami et al., 2007), (Brants et al., 2007), (...",", ,",0.910448,4


In [28]:
df2 = df1.loc[df1['text_clean_len'] < 1000]
df2['text_clean_len'].sort_values()

205      0
694      0
822      0
398      0
307      0
      ... 
984    725
258    743
896    803
561    844
321    851
Name: text_clean_len, Length: 994, dtype: int64